  # Neural Network Training for the actuarial approach

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt 
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json
import csv

### Define the Bounds for Training the Neural Network

Set the range of parameters for training the neural network

In [2]:
N_Iter = 10000 # Number of Iterations
save_interval = 1000 # Save every... iterations in a file
B = 1 # Batch Size

# Bounds for the parameters
ELGD_bounds = 0,0.6
EAD_bounds = 0,1
PD_bounds = 0,0.52
omega_bounds = 0,1
obligor_bounds = 10,100

n_MC = 500000 # Number of Monte Carlo Samples

Apply the training procedure

Inlcude GA approx to the input

In [3]:
transition_matrix = pd.read_csv("transition_matrix_SP.csv",delimiter = ",")
PDs = np.maximum(np.array(transition_matrix["D"])/100,1e-8)

In [4]:
with open("probs.txt") as tweetfile:
    probs = json.load(tweetfile)
probs = {float(k):float(v) for k,v in probs.items()}
PDs =  np.maximum(np.array([float(k) for k,v in probs.items()])/100,1e-10)
PD_distribution = np.array([float(v) for k,v in probs.items()])

In [5]:
for iteration_save in range(6,100):
    List_x = []
    List_y = []
    for iteration in tqdm(range(save_interval)):
        try:

            if (iteration % 2) == 0:
                ELGD,EAD,PD,omega = create_sample_actuarial(ELGD_bounds,EAD_bounds,PD_bounds,omega_bounds,obligor_bounds,B,
                                                           PDs = PDs,PD_distribution=PD_distribution,PDs_Sample = True)

                ELGD[ELGD>0] = 0.1

            elif (iteration % 2) == 1:
                ELGD,EAD,PD,omega = create_sample_actuarial(ELGD_bounds,EAD_bounds,PD_bounds,omega_bounds,obligor_bounds,B,
                                                           PDs = PDs,PD_distribution=PD_distribution,PDs_Sample = True)
                ELGD[ELGD>0] = 0.45

            Input = np.concatenate([ELGD,EAD,PD,omega],axis =1)   

            GA_999 = MC_IS_actuarial(np.reshape(ELGD[0,:],(1,obligor_bounds[1])),
                                                 np.reshape(EAD[0,:],(1,obligor_bounds[1])),
                                                np.reshape(PD[0,:],(1,obligor_bounds[1])),
                                                np.reshape(ELGD[0,:],(1,obligor_bounds[1])),
                                                 q=0.999,n = n_MC)
        except:
            pass # doing nothing on exception
        List_x.append(Input[0])
        List_y.append(GA_999[0])



    pd.DataFrame(List_x).to_csv("csv/X_"+str(iteration_save)+".csv")
    pd.DataFrame(List_y).to_csv("csv/y_"+str(iteration_save)+".csv")

 57%|█████▋    | 574/1000 [20:27:23<16:14:59, 137.32s/it]C:\Users\jul_ses\Anaconda3\envs\tf_cpu\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
 83%|████████▎ | 829/1000 [31:15:22<6:26:50, 135.73s/it] 

KeyboardInterrupt

